In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/daum_movie_review.csv')
# rating이 6보다 작으면 0(부정), 6 이상이면 긍정으로 라벨 생성
y =[0 if rate < 6 else 1 for rate in df.rating]
# 데이터셋을 학습, 검증, 평가 데이터셋으로 분리
X_train_val, X_test, y_train_val, y_test = train_test_split(df. review.tolist(), y, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0)

print('#Train set size:', len(X_train))
print('#Validation set size:', len(X_val))
print('#Test set size:', len(X_test))

#Train set size: 8282
#Validation set size: 2761
#Test set size: 3682


In [2]:
import torch
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))
inputs = tokenizer("안녕하세요. 반갑습니다.")
print(inputs)

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# 토큰화
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")

# Dataset 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

# bert-base-multilingual-cased 사전학습 모형으로부터 분류기 모형을 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',         # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs=2,             # 학습 에포크 수
    evaluation_strategy="steps",    # 에포크마다 검증 데이터셋에 대한 평가 지표를 출력
    eval_steps=500,
    per_device_train_batch_size=8,  # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,              # strength of weight decay
)

# trainer 객체 생성
trainer = Trainer(
    model = model,                  # 학습할 모형
    args=training_args,             # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,    # 학습 데이터셋
    eval_dataset=val_dataset,       # 검증 데이터셋
    compute_metrics=compute_metrics,
)

# 미세조정학습 실행
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-2-ce7cb3001a63>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step,Training Loss,Validation Loss,Accuracy
500,0.551400,0.541830,0.753350
1000,0.504800,0.475738,0.809127
1500,0.436400,0.519934,0.796088
2000,0.414700,0.421388,0.823977


<ipython-input-2-ce7cb3001a63>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
<ipython-input-2-ce7cb3001a63>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
<ipython-input-2-ce7cb3001a63>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
<ipython-input-2-ce7cb3001a63>:17: UserWarning: To copy construct from a tensor, i

TrainOutput(global_step=2072, training_loss=0.4740480005050718, metrics={'train_runtime': 1206.551, 'train_samples_per_second': 13.728, 'train_steps_per_second': 1.717, 'total_flos': 2689808985606720.0, 'train_loss': 0.4740480005050718, 'epoch': 2.0})

In [5]:
trainer.save_model("my_model")

In [7]:
trainer.evaluate(eval_dataset=test_dataset)

<ipython-input-2-ce7cb3001a63>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


{'eval_loss': 0.4463678300380707,
 'eval_accuracy': 0.8147745790331342,
 'eval_runtime': 66.861,
 'eval_samples_per_second': 55.069,
 'eval_steps_per_second': 3.455,
 'epoch': 2.0}